# ARP Two-Moons Classifier #4
Train a small classifier on two-moons with noise=0.08, hidden=32.

In [ ]:
import torch, math, matplotlib.pyplot as plt
from adaptive_dynamics.arp.optimizers import ARP
device='cuda' if torch.cuda.is_available() else 'cpu'

def two_moons(n=560, noise=0.08):
    t = torch.rand(n)*math.pi
    x1 = torch.stack([torch.cos(t), torch.sin(t)],1)
    x2 = torch.stack([1-torch.cos(t), 1-torch.sin(t)-0.5],1)
    X = torch.cat([x1,x2],0) + noise*torch.randn(2*n,2)
    y = torch.cat([torch.zeros(n,dtype=torch.long), torch.ones(n,dtype=torch.long)],0)
    return X,y

X,y = two_moons()
plt.figure(); plt.scatter(X[:,0], X[:,1], c=y, s=8); plt.title('Two Moons'); plt.show()

model = torch.nn.Sequential(
    torch.nn.Linear(2, 32), torch.nn.ReLU(),
    torch.nn.Linear(32, 2)
).to(device)

opt = ARP(model.parameters(), lr=0.006, alpha=0.02, mu=0.002)
loss_fn = torch.nn.CrossEntropyLoss()
X,y = X.to(device), y.to(device)

for epoch in range(1, 61):
    logits = model(X); loss = loss_fn(logits, y)
    loss.backward(); opt.step(); model.zero_grad(set_to_none=True)
    if epoch % 15 == 0:
        acc = (logits.argmax(1)==y).float().mean().item()
        print(f'epoch {epoch} loss={loss.item():.4f} acc={acc:.3f}')